# Correlation (상관관계)
`06_corr.ipynb`

## 정의
- 두 변수간에 서로 얼마나 함께 변하는지 정도를 나타내는 통계 개념
- 값의 범위는 : -1 ~ +1
    - +1: 완벽한 양의 상관관계 -> x가 증가하면 y도 비례해서 증가 : 키 - 몸무게
    - 0: 상관 없음 : 키와 수학점수
    - -1: 완벽한 음의 상관관계 -> x가 증가하면 y가 비례해서 감소 : 운동량 - 체지방률

## 상관 계수 ($r$)
두 변수간의 관계가 얼마나 강한지 측정하는 법 (대표)
- 피어슨 (Pearson): 가장 많이 쓰임. 연속 변수 선형 관계 측정
- 스피어맨 (Spearman): 순위(랭크) 기반, 비 선형 관계, 이상치에 강함
- 켄달 (Kendall) : 순위 일관성 기반, 표본이 적을 때 안정적

In [9]:
%pip install -q kagglehub[pandas-datasets]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


df = pd.read_csv('./OnlineRetail.csv', encoding='ISO-8859-1')

In [11]:
from da_utils import profile, outliers, patterns

# 데이터 프로파일링
profile.get_data_profile(df)

종합 데이터 품질 리포트

1. 기본정보
- 데이터 크기: 541909행 x 8열
- 메모리 사용량: 173.12mb
- 수치형 변수: 3개
- 범주형 변수: 5개

2. 결측값 분석
⚠️


,결측수,결측율(%)
Description,1454,0.27
CustomerID,135080,24.93



3. 수치형 변수 품질 분석


,변수명,평균,표준편차,최솟값,최댓값,왜도,첨도,이상값비율(%),유일값비율(%)
0,Quantity,9.552,218.081,-80995.00,80995.0,-0.264,119769.160,10.817,0.133
1,UnitPrice,4.611,96.760,-11062.06,38970.0,186.507,59005.719,7.312,0.301
2,CustomerID,15287.691,1713.600,12346.00,18287.0,0.030,-1.180,0.000,1.075



4. 분포 이상 감지

- 극심한 왜도 (|skewness|>2)
⚠️
-- UnitPrice: 왜도 = 186.507

- 높은 이상값 비율(>5%)
⚠️
-- Quantity: 이상값 10.8%
-- UnitPrice: 이상값 7.3%


,변수명,평균,표준편차,최솟값,최댓값,왜도,첨도,이상값비율(%),유일값비율(%)
0,Quantity,9.552,218.081,-80995.00,80995.0,-0.264,119769.160,10.817,0.133
1,UnitPrice,4.611,96.760,-11062.06,38970.0,186.507,59005.719,7.312,0.301
2,CustomerID,15287.691,1713.600,12346.00,18287.0,0.030,-1.180,0.000,1.075


In [ ]:
# 결측치 모니터링 함수 
patterns.analyze_missing_patterns(df)

=== 결측값 패턴 분석 ===
변수별 결측 현황


,결측수,결측률(%)
CustomerID,135080,24.93
Description,1454,0.27


In [ ]:
# 이상치 탐지 함수 (결측치 처리후 실행 필요)
outliers.outlier_detection(df, 0.999, iso_cont='auto', final_threshold=2)

In [ ]:
print('전체 거래', len(df))
print('고유 InvoiceNo', df['InvoiceNo'].nunique())
print('고유 고객수', df['CustomerID'].nunique())

In [ ]:
# 데이터 전처리
df_clean = df.copy()

# CustomerID 결측값 제거 (고객 단위 분석을 위해 필수)
missing_customers = df_clean['CustomerID'].isnull().sum()
print(f'CustomerID 결측값 제거: {missing_customers}건')
df_clean = df_clean.dropna(subset=['CustomerID'])

# 취소 거래 분리 (InvoiceNo 가 'C' 로 시작)
cancel_mask = df_clean['InvoiceNo'].astype('str').str.startswith('C')
# 취소거래, 정상거래 분리
df_cancel = df_clean[cancel_mask]
df_clean = df_clean[~cancel_mask]
print(f'취소거래 분리: {len(df_cancel)}건')
print(f'정상 거래: {len(df_clean)}건')

# df_clean에 파생 변수 생성 (상관 관계 분석)
# TotalAmount, InvoiceDate(DateTime), Year, Month, DayOfWeek(숫자), Hour, DayName(글자), CustomerID(Str)
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int).astype(str)
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month']= df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek']= df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['DayName'] = df_clean['InvoiceDate'].dt.day_name()

print('\n=== 이상값 확인 ===')
print(f'음수 수량: {(df_clean['Quantity'] < 0).sum()}')
print(f'음수 단가: {(df_clean['UnitPrice'] < 0).sum()}')
print(f'0 단가: {(df_clean['UnitPrice'] == 0).sum()}')

# 양수 수량 & 양수 단가 데이터만 살리기
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)]

df_clean.info()

In [ ]:
print('=== 기본 상관관계 분석 ===')

# 수치형 상관관계 분석용
numeric_cols =  ['Frequency', '총구매량', '평균구매량', '평균단가', 'Monetary', 
                   '평균구매액', '상품종류수', '구매기간일수', '구매주기', 'Recency', 
                   '평균장바구니크기', '거래당상품종류', '가격변동성']

correlation_data = customer_stats[numeric_cols]
correlation_data.head(3)

In [ ]:
# 피어슨 상관관계 - 키<->몸무게 / 온도<->전력 같이 실제 값이 비례하는 경우 (값)
pearson_corr = correlation_data.corr(method='pearson')
# 스피어만 상관관계 - 시험순위<->대회순위 / 만족도<->재구매의사 같이 서열형 관계에 적합 (순서)
spearman_corr = correlation_data.corr(method='spearman')

fig, axes = plt.subplots(2, 2, figsize=(20, 16))
a1, a2, a3, a4 = axes[0,0], axes[0,1], axes[1,0], axes[1,1]

# 피어슨 상관관계 히트맵    
sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a1, cbar_kws={'label': 'Pearson 상관계수'})
a1.set_title('피어슨 상관관계 (선형 관계)', fontsize=14)

# 스피어만 상관관계 히트맵
sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a2, cbar_kws={'label': 'Spearman 상관계수'})
a2.set_title('스피어만 상관관계 (순위 기반)', fontsize=14)

corr_diff = abs(spearman_corr - pearson_corr)
sns.heatmap(corr_diff, annot=True, fmt='.2f', cmap='Reds',
             square=True, ax=a3, cbar_kws={'label': '|차이|'})
a3.set_title('피어슨 vs 스피어만 차이(비선형성지표)')

# 강한 상관관계 (|r| > 0.5) 네트워크
strong_corr = pearson_corr.copy()
strong_corr[abs(strong_corr) < 0.5] = 0

np.fill_diagonal(strong_corr.values, 0)

sns.heatmap(strong_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a4, cbar_kws={'label': '강한 상관관계'})
a4.set_title('강한 상관관계 (|r| > 0.5)', fontsize=14)
    
plt.tight_layout()
plt.show()


In [ ]:
# 주요 발견사항 요약
print('\n=== 주요 상관관계 발견사항 ===')
# 아래 히트맵에서 위쪽 삼각형만 보겠다
upper_triangle = np.triu(pearson_corr, k=1)
# 가장 큰 값이 있는 인덱스
strong_positive = np.unravel_index(np.argmax(upper_triangle), upper_triangle.shape)
# 가장 강한 양의 상관관계
max_corr = pearson_corr.iloc[strong_positive]
print(f'가장 강한 양의 상관 관계: {max_corr:.3f}')
print(f'  {pearson_corr.index[strong_positive[0]]} <-> {pearson_corr.index[strong_positive[1]]}')

strong_negative = np.unravel_index(np.argmin(upper_triangle), upper_triangle.shape)
min_corr = pearson_corr.iloc[strong_negative]

print(f'가장 강한 음의 상관 관계: {min_corr:.3f}')
print(f'  {pearson_corr.index[strong_negative[0]]} <-> {pearson_corr.index[strong_negative[1]]}')

In [ ]:
print('매출 증대 핵심 요인')
monetary_corr = pearson_corr['Monetary'].abs().sort_values(ascending=False)
for factor, corr in monetary_corr.head(6).items():
    if factor != 'Monetary':
        print(f'  {factor}: {corr:.3f}')


print('\n고객 충성도 관련 요인')
frequency_corr = pearson_corr['Frequency'].abs().sort_values(ascending=False)
for factor, corr in frequency_corr.head(6).items():
    if factor != 'Frequency':
        print(f'  {factor}: {corr:.3f}')


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
# 그래프 2차원 리스트 -> 1차원으로 평탄화
axes = axes.ravel()

relationships = [
    ('평균단가', 'Monetary', '평균단가 vs 총구매액'),
    ('Frequency', '상품종류수', '구매횟수 vs 상품종류수'),
    ('구매주기', 'Recency', '구매주기 vs 최근성'), 
    ('상품종류수', 'Monetary', '상품종류수 vs 총구매액'),
    ('가격변동성', '상품종류수', '가격변동성 vs 상품종류수'),
    ('거래당상품종류', '평균구매액', '거래당상품종류 vs 평균구매액')
]

for idx, (x, y, title) in enumerate(relationships):
    print(customer_stats[x].dtype, customer_stats[y].dtype)
    # 산점도 
    axes[idx].scatter(customer_stats[x], customer_stats[y], alpha=0.6, s=10)
    # 선형 회귀선(트렌드라인)
    z = np.polyfit(customer_stats[x], customer_stats[y], 1)  # 1-> 1차함수로 뽑아라
    print(z[0], z[1])
    # 데이터별 1차함수
    p = np.poly1d(z)
    axes[idx].plot(customer_stats[x], p(customer_stats[x]), 'r--', alpha=0.8, linewidth=2)
    
    # 둘다 높음-> 강한 선형 + 순위
    # P > S -> 값은 비례하지만, 순위는 뒤섞임
    # P < S -> 순위는 일정하지만 비선형관계
    # 둘다 낮음 -> 관계 거의 없음
    pearson_r = customer_stats[x].corr(customer_stats[y], method='pearson')
    spearman_r = customer_stats[x].corr(customer_stats[y], method='spearman')

    axes[idx].set_xlabel(x)
    axes[idx].set_ylabel(y)
    axes[idx].set_title(f'{title}\nP:{pearson_r:.3f}, S:{spearman_r:.3f}')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# 고급 상관관계 측정
print('=== 고급 상관관계 측정 ===')

# 핵심 변수(컬럼)들 선택
key_vars = ['Monetary', 'Frequency', 'Recency', '평균단가', '상품종류수', '평균장바구니크기']
analysis_data = customer_stats[key_vars].copy()

kendall_results = {}

'''
켄달 타우 vs 스피어만 - 둘다 순위 기반
켄달타우: 데이터 개수가 적거나 순위 정보만 확실한 경우, 이상치가 있거나 동점자가 많은 경우
스피어만: 연속형 데이터지만, 선형이 아닐 때, 이상치가 없고 동점자가 적은 경우
'''
from scipy.stats import kendalltau
# 켄달 타우 상관계수 (순위기반 - 이상값에 강함)
print('1. 켄달 타우 상관계수 분석')
for idx, var1 in enumerate(key_vars):
    for var2 in key_vars[idx+1:]:
        tau, p_value = kendalltau(analysis_data[var1], analysis_data[var2])
        # 켄달 타우 결과 확인(유의미한 관계만 -> p_value < 0.05)
        if p_value < 0.05 and abs(tau) > 0.1:
            kendall_results[f'{var1} vs {var2}'] = {'tau': tau, 'p_value': p_value}

for rel, stats in kendall_results.items():
    print(f'  {rel} = {stats['tau']:.3f} (p={stats['p_value']:.3f})')


In [ ]:
# 상호 정보량 (Mutual Information) 기반 분석
# 선형, 비선형 구분없이 변수간 정보량 공유 정도를 측정
# MI = 0 -> 전혀 관련 없음
# MI >= 1 -> 꽤 관련 있음
# MI > 큼 -> 변수 X는 변수Y 예측에 유용하다

# 켄달타우 -> '방향'과 '순위'의 일관성
# MI 점수 -> '관계'가 있긴한데, +- 몰라, 그냥 둘이 강하게 연결 되어있는거냐


from sklearn.feature_selection import mutual_info_regression
print('2. 상호정보량 기반 연관성 (Monetary 기준)')
# Monetary를 타겟으로한 상호 정보량 계산
target = analysis_data['Monetary']
features = analysis_data.drop('Monetary', axis=1)
mi_scores = mutual_info_regression(features, target, random_state=42)
mi_results = pd.DataFrame({
    'Feature': features.columns,
    'MI_Score': mi_scores
}).sort_values('MI_Score', ascending=False)

for _, row in mi_results.iterrows():
    print(f'  {row['Feature']}: {row['MI_Score']:.3f}')

In [ ]:
# 3. 조건부 상관관계 분석 (고객 세그먼트별) - VIP, 일반, 신규

customer_stats['Segment'] = 'Regular'

vip_mask = (
            (customer_stats['R_score'].astype(int) >= 4) &
            (customer_stats['F_score'].astype(int) >= 4) &
            (customer_stats['M_score'].astype(int) >= 4)
        )
customer_stats.loc[vip_mask, 'Segment'] = 'VIP'

new_mask = customer_stats['Frequency'] == 1
customer_stats.loc[new_mask, 'Segment'] = 'New'

segments = ['VIP', 'Regular', 'New']
segment_corrs = {}

for seg in segments:
    seg_data = customer_stats[customer_stats['Segment'] == seg]
    if len(seg_data) > 10:
        # 세그먼트 별로 F <P> M
        corr = seg_data['상품종류수'].corr(seg_data['Monetary'])
        segment_corrs[seg] = {
            'corr': corr,
            'size': len(seg_data)
        }
        print(f'  {seg} 고객: M - F = {corr:.3f}, (n={len(seg_data)})')
    

In [ ]:
s = pd.Series([1, 1, 1, 1, 1, 2, 3, 4])

pd.qcut(s, q=4, duplicates='drop')

In [ ]:
# 비선형 패턴 심화

print('U자형 관계 탐지')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
a1, a2, a3, a4 = axes.ravel()

# 평균 단가 구간별 구매횟수 분석
customer_stats['Price_Segment'] = pd.qcut(customer_stats['평균단가'], q=10, labels=False, duplicates='drop')

price_freq_analysis = customer_stats.groupby('Price_Segment').agg({
    'Frequency': ['mean', 'count'],
    '평균단가': 'mean',
    'Monetary': 'mean'
}).round(2)

price_freq_analysis.columns = ['평균구매횟수', '고객수', '평균단가', '평균총구매액']

a1.plot(price_freq_analysis.index, price_freq_analysis['평균구매횟수'], marker='o', linewidth=2, markersize=8)
a1.set_xlabel('가격 세그먼트(0~9)')
a1.set_ylabel('평균구매횟수')
a1.set_title('가격대별 구매빈도(U자형탐지)')
a1.grid(True, alpha=0.3)

# 구매 상품 다양성 <-> 총구매액
customer_stats['Variety_Segment'] = pd.qcut(customer_stats['상품종류수'], q=10, labels=False, duplicates='drop')

variety_analysis = customer_stats.groupby('Variety_Segment').agg({
    'Monetary': 'mean',
    '평균구매액': 'mean',
    '상품종류수': 'mean',
}).round(2)

a2.plot(variety_analysis['상품종류수'], variety_analysis['Monetary'], marker='s', linewidth=2, markersize=8)
a2.set_xlabel('구매 상품종류 수 평균')
a2.set_ylabel('총구매액 평균')
a2.set_title('상품다양성 vs 총구매액(포화점 탐지)')
a2.grid(True, alpha=0.3)





plt.tight_layout()
plt.show()
